### 환경 불러오기

In [16]:
import pandas as pd
import numpy as np
import os
import requests
import json
# copywarning ignore
import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#### 오피스텔 도로명 주소 Geocoding - 위도, 경도 추출

In [25]:
api_key = '79d3d40921cad65ee45ee1059c974972'

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['road_address']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

def get_lat_lon(df):
    for i in range(0, len(df)):
        try:
            addr = df.iloc[i]['시군구'] + " " + df.iloc[i]['도로명']
            x, y = addr_to_lat_lon(addr)
        except:
            x, y = 0, 0
        df['longitude'].iloc[i] = x
        df['latitude'].iloc[i] = y
    

### 오피스텔 실거래 데이터 불러오기(2022.03 ~ 2023.02)

In [26]:
# officetels 폴더에 있는 모든 파일명 리스트로 저장
off_forders = os.listdir('../data/officetels')
# csv 파일을 모두 불러와서 병합
df_officetels = pd.DataFrame()

for i in range(0, 2):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')
# 도로명 주소가 없는 행은 제거
df_officetels

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\4107615977.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\4107615977.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


In [ ]:
for i in range(2, 4):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


2 complete


C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


3 complete


In [ ]:
for i in range(4, 6):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


4 complete


C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


5 complete


In [ ]:
for i in range(6, 8):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


6 complete


C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


7 complete


In [ ]:
for i in range(8, 10):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


8 complete


C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


9 complete


In [ ]:
for i in range(10, 12):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        sub_df['도로명'].replace(' ',np.nan, inplace=True)
        sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
        sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
        sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
        get_lat_lon(sub_df)
        # concat함수로 행을 추가하며 병합
        sub_df.to_csv(file, encoding='CP949',index=False)
        df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
        print(i,'complete')

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


10 complete


C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'].iloc[i] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_13456\626319627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'].iloc[i] = y


11 complete


In [ ]:
df_officetels.to_csv(path_or_buf='오피스텔 실거래가_22-03_23-02', encoding='CP949')